# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_metadata_path = "../WeatherPy/city_df.csv"

city_metadata_df = pd.read_csv(city_metadata_path)

city_metadata_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Alfred,0,ZA,1611663960,81,-33.5906,26.8910,75.00,7.00
1,Mataura,100,NZ,1611663960,54,-46.1927,168.8643,71.01,8.99
2,Port Elizabeth,20,ZA,1611663960,68,-33.9180,25.5701,73.40,23.02
3,Riberalta,75,BO,1611663804,94,-10.9833,-66.1000,77.00,2.04
4,Punta Arenas,40,CL,1611663804,66,-53.1500,-70.9167,46.40,27.63
...,...,...,...,...,...,...,...,...,...
558,Vestmannaeyjar,75,IS,1611664055,64,63.4427,-20.2734,30.20,34.52
559,Excelsior Springs,90,US,1611664003,86,39.3392,-94.2261,26.60,10.36
560,Sarkand,0,KZ,1611664055,86,45.4100,79.9186,14.49,2.95
561,Raudeberg,100,NO,1611664055,94,61.9875,5.1352,33.01,5.86


In [3]:
city_metadata_df.count()

City          563
Cloudiness    563
Country       560
Date          563
Humidity      563
Lat           563
Lng           563
Max Temp      563
Wind Speed    563
dtype: int64

In [4]:
clean_df = city_metadata_df.dropna(how='any')


In [5]:
clean_df.count()

City          560
Cloudiness    560
Country       560
Date          560
Humidity      560
Lat           560
Lng           560
Max Temp      560
Wind Speed    560
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
clean_df.dtypes

City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [7]:

humidity = clean_df['Humidity']

locations = clean_df[['Lat', 'Lng']]


In [8]:
gmaps.configure(api_key=g_key)

In [9]:
fig=gmaps.figure(center=(50.0, -35.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                  dissipating=False, max_intensity=10, 
                                  point_radius=1)
#heat_layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
** Max Temperature less than 80 degrees but more than 70 degrees
** Wind Speed less than 20 mph
** Less than 20% cloudiness
** Humidity between 30% - 60%
* Drop any rows with null values.

In [10]:
clean_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Alfred,0,ZA,1611663960,81,-33.5906,26.8910,75.00,7.00
1,Mataura,100,NZ,1611663960,54,-46.1927,168.8643,71.01,8.99
2,Port Elizabeth,20,ZA,1611663960,68,-33.9180,25.5701,73.40,23.02
3,Riberalta,75,BO,1611663804,94,-10.9833,-66.1000,77.00,2.04
4,Punta Arenas,40,CL,1611663804,66,-53.1500,-70.9167,46.40,27.63
...,...,...,...,...,...,...,...,...,...
558,Vestmannaeyjar,75,IS,1611664055,64,63.4427,-20.2734,30.20,34.52
559,Excelsior Springs,90,US,1611664003,86,39.3392,-94.2261,26.60,10.36
560,Sarkand,0,KZ,1611664055,86,45.4100,79.9186,14.49,2.95
561,Raudeberg,100,NO,1611664055,94,61.9875,5.1352,33.01,5.86


In [11]:
h_temp = clean_df.loc[clean_df['Max Temp'] <=80]
l_temp =  clean_df.loc[clean_df['Max Temp'] >=70]
wind = clean_df.loc[clean_df['Wind Speed'] <=20]
clouds = clean_df.loc[clean_df['Cloudiness'] <=20]
h_humid =  clean_df.loc[clean_df['Humidity'] <=50]
l_humid =  clean_df.loc[clean_df['Humidity'] >=30]

In [12]:
#merge humidity
merge_humid_df = pd.merge(h_humid, l_humid, how='inner',on=None)

merge_humid_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Launceston,0,AU,1611663962,47,-41.4500,147.1667,55.99,17.27
1,Río Gallegos,40,AR,1611663965,49,-51.6226,-69.2181,48.20,21.85
2,Longyearbyen,0,SJ,1611663966,36,78.2186,15.6401,6.80,32.21
3,Lusambo,75,CD,1611663971,38,-4.9667,23.4500,95.14,2.24
4,Rocha,0,UY,1611663974,50,-34.4833,-54.3333,84.76,6.98
5,Muradnagar,0,IN,1611663975,39,28.7833,77.5000,66.20,5.75
6,Kruisfontein,0,ZA,1611663976,45,-34.0033,24.7314,84.00,11.99
7,Bongandanga,58,CD,1611663976,31,1.5000,21.0500,97.16,3.33
8,Jasrāna,0,IN,1611663978,45,27.2500,78.6833,61.45,5.21
9,Nouadhibou,0,MR,1611663987,43,20.9310,-17.0347,75.20,13.80


In [13]:
#merge_temperature
merge_temp_df = pd.merge(h_temp, l_temp, on=None)

merge_temp_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Alfred,0,ZA,1611663960,81,-33.5906,26.8910,75.00,7.00
1,Mataura,100,NZ,1611663960,54,-46.1927,168.8643,71.01,8.99
2,Port Elizabeth,20,ZA,1611663960,68,-33.9180,25.5701,73.40,23.02
3,Riberalta,75,BO,1611663804,94,-10.9833,-66.1000,77.00,2.04
4,Rikitea,0,PF,1611663961,72,-23.1203,-134.9692,78.91,13.00
...,...,...,...,...,...,...,...,...,...
121,Pochutla,0,MX,1611664053,70,15.7432,-96.4661,70.00,5.01
122,Katherine,75,AU,1611664053,89,-14.4667,132.2667,78.80,10.36
123,Camaná,87,PE,1611664054,71,-16.6228,-72.7111,71.85,7.70
124,Jaciara,59,BR,1611664055,72,-15.9653,-54.9683,78.60,4.25


In [14]:
#merge wind & clouds
merge_wc_df = pd.merge(wind, clouds, on=None)

merge_wc_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Alfred,0,ZA,1611663960,81,-33.5906,26.8910,75.00,7.00
1,Rikitea,0,PF,1611663961,72,-23.1203,-134.9692,78.91,13.00
2,Quatre Cocos,20,MU,1611663962,51,-20.2078,57.7625,87.80,8.05
3,Iqaluit,1,CA,1611663788,77,63.7506,-68.5145,-4.00,16.11
4,Launceston,0,AU,1611663962,47,-41.4500,147.1667,55.99,17.27
...,...,...,...,...,...,...,...,...,...
156,São João da Barra,0,BR,1611664051,51,-21.6403,-41.0511,86.00,13.80
157,Pochutla,0,MX,1611664053,70,15.7432,-96.4661,70.00,5.01
158,Kavaratti,20,IN,1611664054,65,10.5669,72.6420,82.11,7.36
159,Beşikdüzü,0,TR,1611664054,36,41.0520,39.2284,62.60,10.36


In [15]:
merge_wc_temp_df = pd.merge(merge_wc_df, merge_temp_df, on=None)

merge_wc_temp_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Alfred,0,ZA,1611663960,81,-33.5906,26.8910,75.00,7.00
1,Rikitea,0,PF,1611663961,72,-23.1203,-134.9692,78.91,13.00
2,Basco,1,PH,1611663963,77,20.4487,121.9702,72.66,18.03
3,São Filipe,0,CV,1611663963,56,14.8961,-24.4956,76.91,5.14
4,Bredasdorp,18,ZA,1611663967,57,-34.5322,20.0403,77.00,14.97
5,Mawlaik,5,MM,1611663978,52,23.6333,94.4167,70.18,0.92
6,Nouadhibou,0,MR,1611663987,43,20.9310,-17.0347,75.20,13.80
7,Bathsheba,20,BB,1611663992,69,13.2167,-59.5167,78.80,18.41
8,Abu Dhabi,0,AE,1611663971,57,24.4667,54.3667,75.20,11.50
9,Rio Grande da Serra,20,BR,1611663994,83,-23.7442,-46.3983,78.80,5.75


In [16]:
merge_final_df = pd.merge(merge_wc_temp_df, merge_humid_df, on=None)
merge_final_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nouadhibou,0,MR,1611663987,43,20.9310,-17.0347,75.2,13.80
1,Dakar,0,SN,1611663710,36,14.6937,-17.4441,78.8,4.61
2,Faya,0,SA,1611664004,43,18.3851,42.4509,77.0,11.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = merge_final_df

params = {
    'radius': 5000,
    'types':'lodging',
    'key':g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print('Missing result....skipping.')
        
        
    print('------------------')
        
    

Retrieving Results for Index 0: Nouadhibou.
Closest hotel is El Medina.
------------------
Retrieving Results for Index 1: Dakar.
Closest hotel is Novotel Dakar.
------------------
Retrieving Results for Index 2: Faya.
Closest hotel is مقبرة ال مجمل.
------------------


In [18]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Nouadhibou,0,MR,1611663987,43,20.9310,-17.0347,75.2,13.80,El Medina
1,Dakar,0,SN,1611663710,36,14.6937,-17.4441,78.8,4.61,Novotel Dakar
2,Faya,0,SA,1611664004,43,18.3851,42.4509,77.0,11.50,مقبرة ال مجمل


In [19]:
# NOTE: Do not change any of the code in this cell

locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)



In [20]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))